<a href="https://colab.research.google.com/github/bdurantec/company-query-save-spreadsheet/blob/main/CompanyQuery_SaveSpreadsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document number query and insertion into the Google Sheets database
This project aims to populate the CNPJ database by querying data from the Receita API and saving it directly into a Google Sheets database.

## Setup Company Query

In [21]:
import requests
import re
import time

In [29]:
BASE_URL = 'https://receitaws.com.br/v1/cnpj/{}'

In [71]:
def remove_special_char(string):
    return re.sub(r'\D', '', string)

def get_company(document_number, max_retries=3):
    retries = 0
    while retries < max_retries:
        response = requests.get(BASE_URL.format(document_number))

        if response.status_code == 200:
            return response.json()

        elif response.status_code == 429:
            # Too many requests, check if the API provides a retry-after header
            retry_after = int(response.headers.get("Retry-After", 60))
            print(f">> Rate limit exceeded for {document_number}. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1

        else:
            return f"Error for {document_number}: {response.text}"

    return f"Max retries exceeded for {document_number}."

## Setup Google Sheets

In [32]:
!pip install --upgrade gspread gspread_dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
  Attempting uninstall: gspread_dataframe
    Found existing installation: gspread-dataframe 3.3.1
    Uninstalling gspread-dataframe-3.3.1:
      Successfully uninstalled gspread-dataframe-3.3.1


In [61]:
from google.colab import auth
import gspread
from google.auth import default

In [63]:
# Google Auth
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [43]:
# Create a spreadsheet
# sh = gc.create('New spreadsheet')
# sh.share('my-gmail@gmail.com', perm_type='user', role='writer')

## Service

In [72]:
import pandas as pd
from gspread_dataframe import set_with_dataframe

companies = []

def exec(documents, spreadsheet_id):
    get_companies_by_document(documents)
    set_companies_in_spreadsheet(spreadsheet_id)

def get_companies_by_document(documents):
    for document in documents:
        document_number = remove_special_char(document)
        companies.append(get_company(document_number))

    print('>> Document query successfully completed!')

def set_companies_in_spreadsheet(id):
    sh = gc.open_by_key(id)
    df = pd.DataFrame(companies)

    # Select the first tab of the spreadsheet
    worksheet = sh.get_worksheet(0)
    # Save DataFrame in spreadsheet
    set_with_dataframe(worksheet, df)

    print('>> Public company data successfully saved in the Google spreadsheet {}.'.format(id))

## For User

In [76]:
# Insert the CNPJs here
documents = [
    '09.372.158/0001-67',
    '33.014.556/1598-96',
    '32.263.580/0001-04',
    '90.400.888/0001-42'
]

worksheet_id = '1L7NDhpvzH7H3p_rHs0TjCdvviN8YsrqgC6_xSzeih68'

In [74]:
exec(documents, worksheet_id)

>> Rate limit exceeded for 90400888000142. Retrying after 60 seconds...
>> Document query successfully completed!
>> Public company data successfully saved in the Google spreadsheet 1L7NDhpvzH7H3p_rHs0TjCdvviN8YsrqgC6_xSzeih68.


End.